# 사전 준비

In [ ]:
# 코랩 런타임 초기화시 아래 코드 실행
# 이 코드는 패키지를 다시 설치하지 않도록 패키지를 리로드해줍니다.

import os, sys
from google.colab import drive

drive.mount('/content/drive')
my_path = '/content/notebooks'
#os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /usr/local/lib/python3.8/dist-packages/konlpy/java/
os.chdir('/usr/local/lib/python3.8/dist-packages/konlpy/java/dictionary')
!jar xvf ../open-korean-text-2.1.0.jar


/usr/local/lib/python3.8/dist-packages/konlpy/java
java.io.FileNotFoundException: ../open-korean-text-2.1.0.jar (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:219)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:157)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:112)
	at jdk.jartool/sun.tools.jar.Main.run(Main.java:407)
	at jdk.jartool/sun.tools.jar.Main.main(Main.java:1680)


In [ ]:
# # 사전 업데이트

%cd /content/notebooks/konlpy/java/dictionary
!jar cvf open-korean-text-2.1.0.jar *
!mv open-korean-text-2.1.0.jar ../

# 전처리 과정

In [ ]:
# 엑셀 파일 이름 한 번에 꺼내기
# lst_excel 에 저장

%cd /content/drive/MyDrive/crawling_data/ZeroSugar_review_10000


import os

lst_excel = []
dir_path = "/content/drive/MyDrive/crawling_data/ZeroSugar_review_10000"

for (root, directories, files) in os.walk(dir_path):
    for d in directories:
        d_path = os.path.join(root, d)
        print(d_path)
    for file in files:
        file_path = os.path.join(root, file)
        if 'review' in file :
          break
        else :
          # print(file)
          lst_excel.append(file)
# df = pd.read_excel('OneamSparkling_652_Danawa_crawling.xlsx')
# df.head()
# 엑셀 파일 단번에 dataframe으로 가져오기

file_lst = []

import pandas as pd

for exc in lst_excel :
  file_name = exc[:exc.find('_')]
  globals()[file_name] = pd.read_excel(exc)
  print(f'{file_name} excel_pass')
  file_lst.append(file_name)
  print('-'*60)



/content/drive/MyDrive/crawling_data/ZeroSugar_review_10000
/content/drive/MyDrive/crawling_data/ZeroSugar_review_10000/ipynb
Minemine excel_pass
------------------------------------------------------------
WelchsZ excel_pass
------------------------------------------------------------
ChilsungCiderZ excel_pass
------------------------------------------------------------
NobrandSparkling excel_pass
------------------------------------------------------------
PepsiColaZ excel_pass
------------------------------------------------------------
BurrZcola excel_pass
------------------------------------------------------------
Reinwasser excel_pass
------------------------------------------------------------
Seagram excel_pass
------------------------------------------------------------
BurrZcider excel_pass
------------------------------------------------------------
SpriteZ excel_pass
------------------------------------------------------------
Trevi excel_pass
-----------------------------

In [ ]:
file_lst

['Minemine',
 'WelchsZ',
 'ChilsungCiderZ',
 'CocacolaZ',
 'NobrandSparkling',
 'PepsiColaZ',
 'BurrZcola',
 'Reinwasser',
 'Seagram',
 'OneamSparkling',
 'BurrZcider',
 'SpriteZ',
 'Trevi',
 'NarangdCider',
 'Victoria']

In [ ]:
# okt 

# morphs() : 텍스트를 형태소 단위로 나눈다.
# 옵션 : norm, stem. norm은 normalize의 약자로 문장의 정규화 여부를 정하는 옵션이다. 
# stem은 각 단어에서 어간을 추출할지 정하는 옵션이다. 기본값은 둘 다 False.
# nouns() : 텍스트에서 명사만 추출한다.
# phrases() : 텍스트에서 어절을 추출한다.
# pos() : 텍스트를 형태소 단위로 나눈 뒤, 형태소와 품사 정보를 리스트화한 결과를 반환한다.
# pos() 함수도 norm, stem 옵션이 있다.

# okt 연습 실행
# 패키지를 못찾을 경우 경로 설정을 다시해보자
%cd /content/drive/MyDrive/Colab Notebooks/my_env
%cd /content/notebooks
from konlpy.tag import Okt
# norm = True
# norm은 정규화 여부 확인이다. -> 오타를 검증해줌


okt = Okt()
okt.pos('닭가슴살 라임맛 좋고 라임향 힘쌔고 제로슈거 힘듬 배송상태 특유 콜라향 탄산음료 제로칼로리음료' 
        , norm = True , stem = True)

# 중간분류 단어 

In [ ]:
# 문자 정돈

import re

def cleaned_text(text): 
  hangul = re.compile('[^가-힇a-zA-Z0-9]')
  result = hangul.sub(' ', str(text))

  # 띄어쓰기 명사 추가

  pepsi_01 = re.compile('제로 펩시')
  result = pepsi_01.sub('펩시제로', text)

  pepsi_02 = re.compile('펩시 제로')
  result = pepsi_02.sub('펩시제로', text)

  return result

# df['review'] = df['review'].apply(lambda x: cleaned_text(x))
# reviews = df['review']

In [ ]:
def bullword_cleaned_text(text): 
  hangul = re.compile('[^가-힇a-zA-Z0-9]')
  result = hangul.sub('', text)
  return result

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/my_env
from konlpy.tag import Okt
from collections import Counter
from tqdm import tqdm

whole_words = []
# tqdm(range(len(dataset)), desc='단어쌍 만들기 진행중'):
okt = Okt()
with open('/content/drive/MyDrive/crawling_data/one_word.txt' , 'r' , encoding='utf-8') as f :
  file_data = f.readlines()

for name in tqdm(file_lst, desc = f' 단어 수집중')  :
  df = globals()[name]
  df['review'] = df['review'].apply(lambda x: cleaned_text(x))
  reviews = df['review']
  NN_words = []

  for review in reviews :
    okt_pos = okt.pos(review , norm = True , stem = True)

    for word, pos in okt_pos:
      if 'Noun' in pos :
        NN_words.append(word)
      elif 'Adjective' in pos :
        NN_words.append(word)

  one_word = []
  for fi in file_data : 
    one_word.append(bullword_cleaned_text(fi))

  customized_stopwords = one_word

  unique_NN_words = set(NN_words)

  for word in unique_NN_words :
    if word in customized_stopwords:
      while word in NN_words: 
        NN_words.remove(word)
  c = Counter(NN_words)
  #print(c.most_common(50))
  
  list_of_words = []

  for word, count in c.most_common(50):
    list_of_words.append(word)
  whole_words.extend(list_of_words)

/content/drive/MyDrive/Colab Notebooks/my_env


Minemine 단어 수집중: 100%|██████████| 15/15 [16:33<00:00, 66.25s/it]


In [ ]:
c = Counter(whole_words)
hue_word_frm = pd.DataFrame(index = c.keys() , data = c.values() , columns = ['count'])

In [ ]:
hue_word_frm = hue_word_frm.sort_values('count' , ascending = False)
hue_word_frm['percent'] = hue_word_frm['count'] / 15
hue_word_frm

In [ ]:
# okt.pos('맛')
for word in hue_word_frm.query("percent == 1").index :
  for w ,s in okt.pos(word) :
    if s == 'Noun' :
      print(w)
  # print(okt.pos(word))

맛
배송
최고
가격
탄산
구매
마음
제품
재구매
빠르
음료
포장


In [ ]:
# 최고는 제외 
# 맛 , 배송 , 가격 , 탄산
# 4개를 중간분류 단어로 구성

# 데이터 생성
1. 중간분류 단어 탐색
2. 전체 커플 단어 탐색 및 각 커플들의 빈도 수 확인
3. 전체 커플 단어 중 1번 자리의 단어가 중간분류인 단어를 탐색하고 빈도 순서대로 상위 5개를 수집
4. 다시 전체 커플 단어 중 1번 자리의 단어가 3번의 상위 5개 단어와 일치하는지 확인 후 상위 5개를 수집

- 총 : 4 x 5 x 5 100여개 (100개 이상, 이하일 수 있으며 125개를 넘지 않는다.)
- 중복되는 커플 단어들이 있을 수 있다. (상관 없음)

## 전처리 

In [ ]:
import re
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
import networkx as nx
import itertools
from tqdm import tqdm

# 문자 정돈

def cleaned_text(text): 
  hangul = re.compile('[^가-힇a-zA-Z0-9]')
  result = hangul.sub(' ', str(text))

  # 띄어쓰기 명사 추가

  pepsi_01 = re.compile('제로 펩시')
  result = pepsi_01.sub('펩시제로', str(text))

  pepsi_02 = re.compile('펩시 제로')
  result = pepsi_02.sub('펩시제로', str(text))

  return result

def mk_NN_words() :
  df['review'] = df['review'].apply(lambda x: cleaned_text(x))
  reviews = df['review']

  okt = Okt()

  NN_words = []

  for review in reviews:
    okt_pos = okt.pos(review , norm = True , stem = True)

    for word, pos in okt_pos:
      if 'Noun' in pos :
        NN_words.append(word)
      elif 'Adjective' in pos :
        NN_words.append(word)
  return NN_words


# 불용어 사전

def bullword_cleaned_text(text): 
  hangul = re.compile('[^가-힇a-zA-Z0-9]')
  result = hangul.sub('', str(text))
  return result

def bullword_dict() :
  NN_words = mk_NN_words()
  with open('/content/drive/MyDrive/crawling_data/one_word.txt' , 'r' , encoding='utf-8') as f :
    file_data = f.readlines()

  one_word = []
  for fi in file_data : 
    one_word.append(bullword_cleaned_text(fi))

  customized_stopwords = one_word

  unique_NN_words = set(NN_words)

  for word in unique_NN_words :
    if word in customized_stopwords:
      while word in NN_words: 
        NN_words.remove(word)
  return NN_words



def input_data() :
  NN_words = bullword_dict()  
  c = Counter(NN_words)
  # print(c.most_common(50))

  list_of_words = []

  for word, count in c.most_common(100):
    list_of_words.append(word)

  # 노드 생성

  G = nx.Graph()
  G.add_nodes_from(list_of_words)

  # print(G.nodes())
  # print(G.edges())

  # edge 생성

  
  # weight 초기화가 없을 경우 재실행시 계속해서 weight 크기가 커진다.
  weight = 0
  df['review'] = df['review'].apply(lambda x: cleaned_text(x))
  reviews = df['review']

  okt = Okt()
  for review in reviews :

    selected_words = []
    NN_words = []
    # norm = True 정규화 수정
    okt_pos = okt.pos(review , norm = True)

    for word, pos in okt_pos :
      if 'Noun' in pos: # 명사만 가져옴
        NN_words.append(word)
      elif 'Adjective' in pos :
        NN_words.append(word)

    for word in NN_words :
      if word in list_of_words:
        selected_words.append(word)

    selected_words = set(selected_words)

    for pair in list(itertools.combinations(list(selected_words), 2)):
      if pair in G.edges():
        weight = G[pair[0]][pair[1]]['weight']
        weight += 1
        G[pair[0]][pair[1]]['weight'] = weight    
      else:         
        G.add_edge(pair[0], pair[1], weight=1 )
              
  # 생성된 edge 확인해보기
  return nx.get_edge_attributes(G, 'weight')


In [ ]:
# to_dataframe
# 맛 배송 가격 탄산

# 동시 출현 단어 묶음 엑셀로 저장
# 데이터프레임을 엑셀로 저장하여 후에 다시 데이터를 불러올 때 커플데이터 (이하 동시출현 단어)를 재생성하는 일이 없도록 한다.

def couple_data(test_data , name) :
  lst_00 = []
  lst_01 = []
  lst_02 = []
  for g in test_data :
    
    lst_00.append(g[0])
    lst_01.append(g[1])


  x_data = pd.DataFrame(data = lst_00  , columns = ['review01' ])
  x_data['review02'] = lst_01
  x_data['cnt'] = test_data.values()
  couple_data_frm = x_data.sort_values('cnt' , ascending = False).sort_values('review01').reset_index(drop=True)
  
  # to_excel
  couple_data_frm.to_excel(f'/content/drive/MyDrive/crawling_data/couple_frm/{name}_couple_frm.xlsx' , index = False)

  return couple_data_frm


## 동시 출현 단어가 묶여있는 input data의 데이터 프레임 생성
- ex) [(a,b) , (c,d) ,...]
- 실제 사용 (x)
- 구조 확인용

In [ ]:
# excel 저장용
# 동시 출현 단어의 'ex) 예시 (x, y) x가 부모' 부모가 맛 배송 가격 탄산인 항목만 수집
# 하위 노드들은 상위 노드들과 겹치지 않도록 쿼리 문에서 조절


def cp_input(couple_data_frm , obj_name) :
  data_lst =  []
  for conec_01 in ['맛' , '배송' , '가격' , '탄산'] :
    globals()[f'{conec_01}_frm'] = couple_data_frm.query("review01 == @conec_01 and review02 not in ['맛' , '배송' , '가격' , '탄산'] and review02 not in @obj_name").sort_values('cnt' , ascending = False)[:3].reset_index(drop=True)

    for rv_01 , rv_02 , cnt in zip(globals()[f'{conec_01}_frm'].review01 , globals()[f'{conec_01}_frm'].review02 , globals()[f'{conec_01}_frm'].cnt) :
      data_lst.append([rv_01 , rv_02])

    for conec_02 in globals()[f'{conec_01}_frm'].review02 :
      globals()[f'{conec_02}_frm'] = couple_data_frm.query("review01 == @conec_02 and review02 not in @obj_name").sort_values('cnt' , ascending = False)[:3].reset_index(drop=True)
      
      for rv_01 , rv_02 , cnt in zip(globals()[f'{conec_02}_frm'].review01 , globals()[f'{conec_02}_frm'].review02 , globals()[f'{conec_02}_frm'].cnt) :
        data_lst.append([rv_01 , rv_02])  

  return data_lst

## 실제 input data 생성 함수
- 동시 출현 단어 데이터 프레임을 불러오고 해당 프레임의 상위 노드와 하위 노드가 서로 겹치지 않도록 조건을 주면서 반복한다.

In [ ]:
# input data 생성
# couple_data_frm
# 500~ 625개 값 
# ['맛' , '배송' , '가격' , '탄산']
# ['맛','배송','최고','가격','탄산','구매','마음','제품','재구매','빠르다','음료','포장']
def cp_input_triple(couple_data_frm , name) :
  obj_name = ['미네마인', '빅토리아', '웰치스', '칠성사이다', '코카콜라','노브랜드', '트레비', '펩시콜라', '부르르', '라인바싸', '씨그램', '원에이엠', '나랑드', '스프라이트']
  print()
  print(name)
  print('맛 : 0 , 가격 : 1  , 탄산 : 2 , 배송 : 3')
  print()
  seet_lst =  []
  
  # 각 제품에서 모두 출현하는 단어들

  best_word = ['맛','배송','최고','가격','탄산','구매','마음','제품','재구매','빠르다','음료','포장']

  # 맛 , 가격 , 탄산 , 배송 각각의 그래프를 생성해야함

  for conec_01 in ['맛' , '가격' , '탄산' , '배송'] :
    
    # 아래의 lst는 하위 노드들이 중복되지 않도록 이미 등장한 노드들을 저장하고 반복문에서 검사하는 용도이다.

    # conec_01 word 모음
    conec_01_word = []
    # conec_02 word 모음
    conec_02_word = []
    # conec_03 word 모음
    conec_03_word = []

    data_lst =  []
    globals()[f'{conec_01}_frm'] = couple_data_frm.query(
        "review01 == @conec_01 and review02 not in @best_word and review02 not in @obj_name and review02 not in @conec_01_word and review02 not in @conec_02_word and review02 not in @conec_03_word"
        ).sort_values('cnt' , ascending = False)[:3].reset_index(drop=True)
     # display(globals()[f'{conec_01}_frm'])

    for rv_01 , rv_02 in zip(globals()[f'{conec_01}_frm'].review01 , globals()[f'{conec_01}_frm'].review02) :
      data_lst.append([rv_01 , rv_02])
      conec_01_word.append(rv_02)
      # print('1번째 뿌리\n')
      # print(len(data_lst))
      
    # print()  
    print(f'{conec_01} 시작')
    print(conec_01_word)
    # print('-'*100)
    # print()

    # 나랑드사이다 - 맛 연결 키워드
    for conec_02 in globals()[f'{conec_01}_frm'].review02 :
      globals()[f'{conec_02}_frm'] = couple_data_frm.query(
          "review01 == @conec_02 and review02 not in @obj_name and review02 not in @conec_01 and review02 not in @conec_01_word and review02 not in @conec_02_word and review02 not in @conec_03_word"
      ).sort_values('cnt' , ascending = False)[:3].reset_index(drop=True)
      
      for rv_01 , rv_02 in zip(globals()[f'{conec_02}_frm'].review01 , globals()[f'{conec_02}_frm'].review02) :
        data_lst.append([rv_01 , rv_02])  
        conec_02_word.append(rv_02)
        # print('2번째 뿌리\n')  
        # print(len(data_lst))
      
      # print('-'*100)

      # print(f'{conec_02}_frm')
      # display(globals()[f'{conec_02}_frm'])
          # 나랑드사이다 - 맛 - 칼로리 연결 키워드
      for conec_03 in globals()[f'{conec_02}_frm'].review02 :
        globals()[f'{conec_03}_frm'] = couple_data_frm.query(
            "review01 == @conec_03 and review02 not in @obj_name and review02 not in @conec_01 and review02 not in @conec_01_word and review02 not in @conec_02_word and review02 not in @conec_03_word"
            ).sort_values('cnt' , ascending = False)[:2].reset_index(drop=True)
        for rv_01 , rv_02 in zip(globals()[f'{conec_03}_frm'].review01 , globals()[f'{conec_03}_frm'].review02) :
          data_lst.append([rv_01 , rv_02])     
          conec_03_word.append(rv_02)

          # print('3번째 뿌리\n')  
          # print(len(data_lst))
        # print(f'{conec_03}_frm')
        # display(globals()[f'{conec_03}_frm'])

    seet_lst.append(data_lst)
      # print('-'*100)
      # print('*'*100)
  return seet_lst

## 실행문

In [ ]:
# 각 제품들의 데이터프레임 변수 명이 담긴 리스트

file_lst

['Minemine',
 'WelchsZ',
 'ChilsungCiderZ',
 'CocacolaZ',
 'NobrandSparkling',
 'PepsiColaZ',
 'BurrZcola',
 'Reinwasser',
 'Seagram',
 'OneamSparkling',
 'BurrZcider',
 'SpriteZ',
 'Trevi',
 'NarangdCider',
 'Victoria']

In [ ]:
# 커플 데이터프레임 저장
# couple_data() 실행시 자동으로 저장된다.

obj_name = ['미네마인', '빅토리아', '웰치스', '칠성사이다', '코카콜라','노브랜드', '트레비', '펩시콜라', '부르르', '라인바싸', '씨그램', '원에이엠', '나랑드', '스프라이트']
all_input_data = []
for name in tqdm(file_lst, desc = f'input data 수집중')  :
  df = globals()[name]
  test_data = input_data()
  couple_data(test_data , name)

input data 수집중: 100%|██████████| 15/15 [23:23<00:00, 93.58s/it]


In [ ]:
# 제품별 커플 데이터(이하 동시 출현 단어)들을 불러오고 그래프 input 데이터를 만들어준다.

from tqdm import tqdm 

view_lst = []

for name in tqdm(file_lst, desc = f'input data 수집중')  :
  print(f'{name}----------------------------------------------------------------------------------')
  cp_data = pd.read_excel(f'/content/drive/MyDrive/crawling_data/couple_frm/{name}_couple_frm.xlsx')
  input = cp_input_triple(cp_data , name)

  view_lst.append(input)
  print()
  print()


In [ ]:
# 그래프 input data가 담긴 변수다

view_lst

In [ ]:
# 커플이 따로 노는 input 데이터 프레임 엑셀 변환
# 따로 논다는 뜻은 input data의 동시 출현 단어가 () 로 묶여있지 않고 각각의 칼럼으로 분류되어 있다는 뜻이다.

review_01_lst = []
review_02_lst = []

for value_num, value in enumerate(file_lst) :
  globals()[f'{value}_input_DataFrame'] = pd.DataFrame()
  for middle_num, middle in enumerate(['맛','가격','탄산','배송']) :
    review_01_lst = []
    review_02_lst = []
    for words in view_lst[value_num][middle_num] :
      review_01_lst.append(words[0])
      review_02_lst.append(words[1])
    if len(globals()[f'{value}_input_DataFrame']) > len(review_01_lst) :
      for null_cnt in range(len(globals()[f'{value}_input_DataFrame']) - len(review_01_lst)) :
        review_01_lst.append(None)
        review_02_lst.append(None)
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_01'] = review_01_lst
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_02'] = review_02_lst
    elif len(globals()[f'{value}_input_DataFrame']) < len(review_01_lst) :
      for null_cnt in range(len(review_01_lst) - len(globals()[f'{value}_input_DataFrame'])) :
        globals()[f'{value}_input_DataFrame'] = globals()[f'{value}_input_DataFrame'].append({'맛_review_01' : 0 , '맛_review_02' : 0 },ignore_index=True)
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_01'] = review_01_lst
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_02'] = review_02_lst
    else :
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_01'] = review_01_lst
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_02'] = review_02_lst




In [ ]:
#review_01_lst
for disp in file_lst : 
  display(globals()[f'{disp}_input_DataFrame'])

,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,레몬,가격,동원,탄산,탄산수,배송,탄산수
1,맛,플레인,가격,레몬,탄산,생수,배송,기사
2,맛,처음,가격,스파클링,탄산,가성비,배송,유통기한
3,레몬,라임,동원,워터,탄산수,맛,탄산수,맛
4,레몬,청,동원,처음,탄산수,가격,탄산수,가격
5,레몬,품절,동원,정제수,탄산수,동원,탄산수,동원
6,라임,생각,워터,품절,맛,레몬,맛,레몬
7,라임,도착,워터,비닐,맛,빠르다,맛,빠르다
8,청,과일,처음,라임,가격,제품,가격,제품
9,청,에이드,처음,느낌,가격,재구매,가격,재구매


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,컵,가격,부담없음,탄산,느낌,배송,사은품
1,맛,포도,가격,생각,탄산,칼로리,배송,포도
2,맛,칼로리,가격,걱정,탄산,생각,배송,도착
3,컵,구매,부담없음,원래,느낌,밍밍,사은품,마음
4,컵,사은품,부담없음,느낌,느낌,단맛,사은품,이뻐
5,컵,음료,부담없음,변색,느낌,살짝,사은품,빠르다
6,구매,오렌지,원래,특유,밍밍,끝맛,마음,변색
7,구매,제품,원래,살짝,밍밍,오리,마음,차이
8,사은품,마음,느낌,밍밍,단맛,특유,이뻐,유리컵
9,사은품,이뻐,느낌,단맛,단맛,그대로,이뻐,끝맛


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,칼로리,가격,쿠폰,탄산,칼로리,배송,박스
1,맛,차이,가격,만족,탄산,차이,배송,스마일
2,맛,콜라,가격,할인,탄산,콜라,배송,저렴하다
3,칼로리,부담없음,쿠폰,사용,칼로리,부담없음,박스,캔
4,칼로리,걱정,쿠폰,유통기한,칼로리,걱정,박스,도착
5,칼로리,음료,쿠폰,스마일,칼로리,음료,박스,할인
6,부담없음,할인,유통기한,상태,부담없음,할인,캔,처음
7,부담없음,설탕,유통기한,기존,부담없음,설탕,캔,냉장고
8,걱정,당,스마일,하루,걱정,당,도착,택배
9,걱정,탄산음료,스마일,롯데,걱정,탄산음료,도착,파손


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,칼로리,가격,할인,탄산,칼로리,배송,콜라
1,맛,유통기한,가격,캔,탄산,부담없음,배송,제로
2,맛,캔,가격,유통기한,탄산,느낌,배송,제로콜라
3,칼로리,부담없음,할인,쿠폰,칼로리,걱정,콜라,맛
4,칼로리,걱정,할인,행사,칼로리,유통기한,콜라,캔
5,칼로리,마음,할인,부담없음,칼로리,마음,콜라,가격
6,부담없음,탄산음료,쿠폰,마트,걱정,처음,맛,탄산
7,부담없음,설탕,쿠폰,이용,걱정,설탕,맛,칼로리
8,걱정,처음,행사,아들,유통기한,도착,캔,박스
9,걱정,당,행사,쟁여두,유통기한,파손,캔,포장


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,칼로리,가격,품질,탄산,라임,배송,이용
1,맛,사이다,가격,스파클링,탄산,단맛,배송,가성비
2,맛,라임,가격,사이다,탄산,제로,배송,이마트
3,칼로리,제로,품질,갑,라임,스파클링,이용,쓱배송
4,칼로리,탄산,품질,만족,라임,망고,이용,마트
5,칼로리,부담없음,품질,무난,라임,상큼,이용,종종
6,제로,탄산음료,만족,용량,스파클링,음료,쓱배송,집
7,제로,최고,만족,청포도,스파클링,품절,쓱배송,청포도
8,탄산,가격,무난,리터,망고,오렌지,마트,비대면
9,탄산,단맛,무난,사용,망고,포도,마트,수고


...

,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,제로,가격,나랑드사이다,탄산,칼로리,배송,박스
1,맛,칼로리,가격,칼로리,탄산,단맛,배송,캔
2,맛,나랑드사이다,가격,콜라,탄산,나랑드사이다,배송,도착
3,제로,탄산,나랑드사이다,단맛,칼로리,부담없음,박스,포장
4,제로,구매,나랑드사이다,개인,칼로리,음료,박스,음료
5,제로,음료,나랑드사이다,박스,칼로리,재구매,박스,택배
6,탄산,가격,단맛,특유,부담없음,다이어트,포장,부담없음
7,탄산,단맛,단맛,탄산수,부담없음,탄산음료,포장,상태
8,구매,박스,개인,브랜드,음료,최고,음료,최고
9,구매,제품,개인,별로,음료,설탕,음료,단맛


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,사이다,가격,칼로리,탄산,부담없음,배송,캔
1,맛,칼로리,가격,사은품,탄산,생각,배송,문제
2,맛,차이,가격,사이다,탄산,콜라,배송,박스
3,사이다,콜라,칼로리,부담없음,부담없음,탄산음료,캔,포장
4,사이다,제로콜라,칼로리,걱정,부담없음,설탕,캔,음료
5,사이다,탄산,칼로리,탄산,부담없음,슈가,캔,바닥
6,콜라,박스,부담없음,탄산음료,탄산음료,다이어트,포장,비닐
7,콜라,처음,부담없음,설탕,탄산음료,탄산수,포장,사은품
8,제로콜라,생각,걱정,냉장고,설탕,단맛,음료,최고
9,제로콜라,단맛,걱정,살짝,설탕,걱정,음료,처음


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,자몽,가격,품질,탄산,여름,배송,저렴하다
1,맛,레몬,가격,마트,탄산,사이즈,배송,가성비
2,맛,용량,가격,이마트,탄산,콜라,배송,감사
3,자몽,에이드,품질,양도,여름,필수,저렴하다,가격
4,자몽,플레인,품질,추천,여름,음료,저렴하다,빠르다
5,자몽,입맛,품질,마음,여름,청,저렴하다,포장
6,에이드,집,양도,맥주,필수,구비,가격,품질
7,에이드,과일,양도,부담없음,필수,간식,가격,마트
8,플레인,품절,추천,사용,음료,집,빠르다,재구매
9,플레인,상큼,추천,필수,음료,최고,빠르다,도착


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,칼로리,가격,칼로리,탄산,다이어트,배송,나랑드사이다
1,맛,제로,가격,사용,탄산,캔,배송,사이다
2,맛,사용,가격,도움,탄산,부담없음,배송,칼로리
3,칼로리,부담없음,칼로리,부담없음,다이어트,탄산음료,나랑드사이다,제로
4,칼로리,탄산,칼로리,탄산,다이어트,최고,나랑드사이다,맛
5,칼로리,걱정,칼로리,걱정,다이어트,필수,나랑드사이다,탄산
6,부담없음,설탕,부담없음,설탕,탄산음료,설탕,제로,부담없음
7,부담없음,탄산음료,부담없음,탄산음료,탄산음료,걱정,제로,음료
8,탄산,다이어트,탄산,다이어트,최고,탄산수,맛,포장
9,탄산,음료,탄산,음료,최고,도움,맛,재구매


,맛_review_01,맛_review_02,가격_review_01,가격_review_02,탄산_review_01,탄산_review_02,배송_review_01,배송_review_02
0,맛,청포도,가격,저렴하다,탄산,청포도,배송,박스
1,맛,파인애플,가격,청포도,탄산,탄산음료,배송,청포도
2,맛,레몬,가격,박스,탄산,콜라,배송,처음
3,청포도,복숭아,저렴하다,마음,청포도,파인애플,박스,포장
4,청포도,자몽,저렴하다,최고,청포도,복숭아,박스,파인애플
5,청포도,처음,저렴하다,음료,청포도,레몬,박스,택배
6,복숭아,배,마음,느낌,파인애플,자몽,포장,라임
7,복숭아,개인,마음,상큼,파인애플,배,포장,도착
8,자몽,라임,최고,가성비,복숭아,개인,파인애플,복숭아
9,자몽,플레인,최고,파인애플,복숭아,여러가지,파인애플,자몽


In [ ]:
for disp in file_lst : 
  for colu in Minemine_input_DataFrame.columns :
    globals()[f'{disp}_input_DataFrame'].colu

In [ ]:
Minemine_input_DataFrame['맛_review_01']

0        맛
1        맛
2        맛
3       레몬
4       레몬
5       레몬
6       라임
7       라임
8        청
9        청
10      품절
11      품절
12     플레인
13     플레인
14     플레인
15     정제수
16     정제수
17    유통기한
18    유통기한
19      샘물
20      처음
21      처음
22      처음
23    탄산음료
24    탄산음료
25      워터
26      워터
27      포장
28      포장
29       0
Name: 맛_review_01, dtype: object

In [ ]:
# 커플이 따로 노는 input 데이터 프레임 엑셀 변환

review_01_lst = []
review_02_lst = []

for value_num, value in enumerate(file_lst) :
  print(value)
  print() 
  globals()[f'{value}_input_DataFrame'] = pd.DataFrame()
  for middle_num, middle in enumerate(['맛','가격','탄산','배송']) :
    review_01_lst = []
    review_02_lst = []
    for words in view_lst[value_num][middle_num] :
      review_01_lst.append(words[0])
      review_02_lst.append(words[1])
    if len(globals()[f'{value}_input_DataFrame']) > len(review_01_lst) :
      for null_cnt in range(len(globals()[f'{value}_input_DataFrame']) - len(review_01_lst)) :
        review_01_lst.append(None)
        review_02_lst.append(None)
        print(review_01_lst)
        print(review_02_lst)
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_01'] = review_01_lst
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_02'] = review_02_lst
    elif len(globals()[f'{value}_input_DataFrame']) < len(review_01_lst) :
      for null_cnt in range(len(review_01_lst) - len(globals()[f'{value}_input_DataFrame'])) :
        globals()[f'{value}_input_DataFrame'] = globals()[f'{value}_input_DataFrame'].append({'맛_review_01' : 0 , '맛_review_02' : 0 },ignore_index=True)
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_01'] = review_01_lst
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_02'] = review_02_lst
      print(review_01_lst)
      print(review_02_lst)      
    else :
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_01'] = review_01_lst
      globals()[f'{value}_input_DataFrame'][f'{middle}_review_02'] = review_02_lst
      print(review_01_lst)
      print(review_02_lst)
  print()  
  print() 
  print() 




Minemine

['맛', '맛', '맛', '레몬', '레몬', '레몬', '라임', '라임', '청', '청', '품절', '품절', '플레인', '플레인', '플레인', '정제수', '정제수', '유통기한', '유통기한', '샘물', '처음', '처음', '처음', '탄산음료', '탄산음료', '워터', '워터', '포장', '포장']
['레몬', '플레인', '처음', '라임', '청', '품절', '생각', '도착', '과일', '에이드', '쟁여두', '브랜드', '정제수', '유통기한', '샘물', '사용', '편이', '택배', '느낌', '할인', '탄산음료', '워터', '포장', '음료', '콜라', '비닐', '수고', '박스', '물']
['가격', '가격', '가격', '동원', '동원', '동원', '워터', '워터', '처음', '처음', '정제수', '정제수', '레몬', '레몬', '레몬', '플레인', '플레인', '청', '청', '여름', '여름', '스파클링', '스파클링', '스파클링', '포장', '포장', '택배', '택배', '추천', '추천']
['동원', '레몬', '스파클링', '워터', '처음', '정제수', '품절', '비닐', '라임', '느낌', '샘물', '사용', '플레인', '청', '여름', '유통기한', '에이드', '과일', '음료', '냉장고', '물', '포장', '택배', '추천', '박스', '도착', '기사', '감사', '생각', '청량감']
['탄산', '탄산', '탄산', '탄산수', '탄산수', '탄산수', '맛', '맛', '가격', '가격', '동원', '동원', '생수', '생수', '생수', '정제수', '정제수', '마음', '마음', '기사', '기사', '가성비', '가성비', '가성비', '최고', '최고', '짱', '짱', '플레인', '플레인']
['탄산수', '생수', '가성비', '맛', '가격', '동원', '레몬', '빠르다', '제품', '재구매

In [ ]:
##### 사용하지 않는다.

# # 커플이 함께 노는 input 데이터 프레임 엑셀 변환

# for value_num, value in enumerate(file_lst) :
#   globals()[f'{value}_input_DataFrame'] = pd.DataFrame()
#   for middle_num, middle in enumerate(['맛','가격','탄산','배송']) :
#     review_lst = []
#     for words in view_lst[value_num][middle_num] :
#       review_lst.append([words[0] , words[1]])
#     if len(globals()[f'{value}_input_DataFrame']) > len(review_lst) :
#       for null_cnt in range(len(globals()[f'{value}_input_DataFrame']) - len(review_lst)) :
#         review_lst.append(None)
#       globals()[f'{value}_input_DataFrame'][f'{middle}_review'] = review_lst
#     elif len(globals()[f'{value}_input_DataFrame']) < len(review_lst) :
#       for null_cnt in range(len(review_lst) - len(globals()[f'{value}_input_DataFrame'])) :
#         globals()[f'{value}_input_DataFrame'] = globals()[f'{value}_input_DataFrame'].append({'맛_review' : None },ignore_index=True)
#       globals()[f'{value}_input_DataFrame'][f'{middle}_review'] = review_lst
#     else :globals()[f'{value}_input_DataFrame'][f'{middle}_review'] = review_lst


In [ ]:
# #review_01_lst
# for disp in file_lst : 
#   display(globals()[f'{disp}_input_DataFrame'])

,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 레몬]","[가격, 동원]","[탄산, 탄산수]","[배송, 탄산수]"
1,"[맛, 플레인]","[가격, 레몬]","[탄산, 생수]","[배송, 기사]"
2,"[맛, 처음]","[가격, 스파클링]","[탄산, 가성비]","[배송, 유통기한]"
3,"[레몬, 라임]","[동원, 워터]","[탄산수, 맛]","[탄산수, 맛]"
4,"[레몬, 청]","[동원, 처음]","[탄산수, 가격]","[탄산수, 가격]"
5,"[레몬, 품절]","[동원, 정제수]","[탄산수, 동원]","[탄산수, 동원]"
6,"[라임, 생각]","[워터, 품절]","[맛, 레몬]","[맛, 레몬]"
7,"[라임, 도착]","[워터, 비닐]","[맛, 빠르다]","[맛, 빠르다]"
8,"[청, 과일]","[처음, 라임]","[가격, 제품]","[가격, 제품]"
9,"[청, 에이드]","[처음, 느낌]","[가격, 재구매]","[가격, 재구매]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 컵]","[가격, 부담없음]","[탄산, 느낌]","[배송, 사은품]"
1,"[맛, 포도]","[가격, 생각]","[탄산, 칼로리]","[배송, 포도]"
2,"[맛, 칼로리]","[가격, 걱정]","[탄산, 생각]","[배송, 도착]"
3,"[컵, 구매]","[부담없음, 원래]","[느낌, 밍밍]","[사은품, 마음]"
4,"[컵, 사은품]","[부담없음, 느낌]","[느낌, 단맛]","[사은품, 이뻐]"
5,"[컵, 음료]","[부담없음, 변색]","[느낌, 살짝]","[사은품, 빠르다]"
6,"[구매, 오렌지]","[원래, 특유]","[밍밍, 끝맛]","[마음, 변색]"
7,"[구매, 제품]","[원래, 살짝]","[밍밍, 오리]","[마음, 차이]"
8,"[사은품, 마음]","[느낌, 밍밍]","[단맛, 특유]","[이뻐, 유리컵]"
9,"[사은품, 이뻐]","[느낌, 단맛]","[단맛, 그대로]","[이뻐, 끝맛]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 칼로리]","[가격, 쿠폰]","[탄산, 칼로리]","[배송, 박스]"
1,"[맛, 차이]","[가격, 만족]","[탄산, 차이]","[배송, 스마일]"
2,"[맛, 콜라]","[가격, 할인]","[탄산, 콜라]","[배송, 저렴하다]"
3,"[칼로리, 부담없음]","[쿠폰, 사용]","[칼로리, 부담없음]","[박스, 캔]"
4,"[칼로리, 걱정]","[쿠폰, 유통기한]","[칼로리, 걱정]","[박스, 도착]"
5,"[칼로리, 음료]","[쿠폰, 스마일]","[칼로리, 음료]","[박스, 할인]"
6,"[부담없음, 할인]","[유통기한, 상태]","[부담없음, 할인]","[캔, 처음]"
7,"[부담없음, 설탕]","[유통기한, 기존]","[부담없음, 설탕]","[캔, 냉장고]"
8,"[걱정, 당]","[스마일, 하루]","[걱정, 당]","[도착, 택배]"
9,"[걱정, 탄산음료]","[스마일, 롯데]","[걱정, 탄산음료]","[도착, 파손]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 칼로리]","[가격, 할인]","[탄산, 칼로리]","[배송, 콜라]"
1,"[맛, 유통기한]","[가격, 캔]","[탄산, 부담없음]","[배송, 제로]"
2,"[맛, 캔]","[가격, 유통기한]","[탄산, 느낌]","[배송, 제로콜라]"
3,"[칼로리, 부담없음]","[할인, 쿠폰]","[칼로리, 걱정]","[콜라, 맛]"
4,"[칼로리, 걱정]","[할인, 행사]","[칼로리, 유통기한]","[콜라, 캔]"
5,"[칼로리, 마음]","[할인, 부담없음]","[칼로리, 마음]","[콜라, 가격]"
6,"[부담없음, 탄산음료]","[쿠폰, 마트]","[걱정, 처음]","[맛, 탄산]"
7,"[부담없음, 설탕]","[쿠폰, 이용]","[걱정, 설탕]","[맛, 칼로리]"
8,"[걱정, 처음]","[행사, 아들]","[유통기한, 도착]","[캔, 박스]"
9,"[걱정, 당]","[행사, 쟁여두]","[유통기한, 파손]","[캔, 포장]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 칼로리]","[가격, 품질]","[탄산, 라임]","[배송, 이용]"
1,"[맛, 사이다]","[가격, 스파클링]","[탄산, 단맛]","[배송, 가성비]"
2,"[맛, 라임]","[가격, 사이다]","[탄산, 제로]","[배송, 이마트]"
3,"[칼로리, 제로]","[품질, 갑]","[라임, 스파클링]","[이용, 쓱배송]"
4,"[칼로리, 탄산]","[품질, 만족]","[라임, 망고]","[이용, 마트]"
5,"[칼로리, 부담없음]","[품질, 무난]","[라임, 상큼]","[이용, 종종]"
6,"[제로, 탄산음료]","[만족, 용량]","[스파클링, 음료]","[쓱배송, 집]"
7,"[제로, 최고]","[만족, 청포도]","[스파클링, 품절]","[쓱배송, 청포도]"
8,"[탄산, 가격]","[무난, 리터]","[망고, 오렌지]","[마트, 비대면]"
9,"[탄산, 단맛]","[무난, 사용]","[망고, 포도]","[마트, 수고]"


...

,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 제로]","[가격, 나랑드사이다]","[탄산, 칼로리]","[배송, 박스]"
1,"[맛, 칼로리]","[가격, 칼로리]","[탄산, 단맛]","[배송, 캔]"
2,"[맛, 나랑드사이다]","[가격, 콜라]","[탄산, 나랑드사이다]","[배송, 도착]"
3,"[제로, 탄산]","[나랑드사이다, 단맛]","[칼로리, 부담없음]","[박스, 포장]"
4,"[제로, 구매]","[나랑드사이다, 개인]","[칼로리, 음료]","[박스, 음료]"
5,"[제로, 음료]","[나랑드사이다, 박스]","[칼로리, 재구매]","[박스, 택배]"
6,"[탄산, 가격]","[단맛, 특유]","[부담없음, 다이어트]","[포장, 부담없음]"
7,"[탄산, 단맛]","[단맛, 탄산수]","[부담없음, 탄산음료]","[포장, 상태]"
8,"[구매, 박스]","[개인, 브랜드]","[음료, 최고]","[음료, 최고]"
9,"[구매, 제품]","[개인, 별로]","[음료, 설탕]","[음료, 단맛]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 사이다]","[가격, 칼로리]","[탄산, 부담없음]","[배송, 캔]"
1,"[맛, 칼로리]","[가격, 사은품]","[탄산, 생각]","[배송, 문제]"
2,"[맛, 차이]","[가격, 사이다]","[탄산, 콜라]","[배송, 박스]"
3,"[사이다, 콜라]","[칼로리, 부담없음]","[부담없음, 탄산음료]","[캔, 포장]"
4,"[사이다, 제로콜라]","[칼로리, 걱정]","[부담없음, 설탕]","[캔, 음료]"
5,"[사이다, 탄산]","[칼로리, 탄산]","[부담없음, 슈가]","[캔, 바닥]"
6,"[콜라, 박스]","[부담없음, 탄산음료]","[탄산음료, 다이어트]","[포장, 비닐]"
7,"[콜라, 처음]","[부담없음, 설탕]","[탄산음료, 탄산수]","[포장, 사은품]"
8,"[제로콜라, 생각]","[걱정, 냉장고]","[설탕, 단맛]","[음료, 최고]"
9,"[제로콜라, 단맛]","[걱정, 살짝]","[설탕, 걱정]","[음료, 처음]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 자몽]","[가격, 품질]","[탄산, 여름]","[배송, 저렴하다]"
1,"[맛, 레몬]","[가격, 마트]","[탄산, 사이즈]","[배송, 가성비]"
2,"[맛, 용량]","[가격, 이마트]","[탄산, 콜라]","[배송, 감사]"
3,"[자몽, 에이드]","[품질, 양도]","[여름, 필수]","[저렴하다, 가격]"
4,"[자몽, 플레인]","[품질, 추천]","[여름, 음료]","[저렴하다, 빠르다]"
5,"[자몽, 입맛]","[품질, 마음]","[여름, 청]","[저렴하다, 포장]"
6,"[에이드, 집]","[양도, 맥주]","[필수, 구비]","[가격, 품질]"
7,"[에이드, 과일]","[양도, 부담없음]","[필수, 간식]","[가격, 마트]"
8,"[플레인, 품절]","[추천, 사용]","[음료, 집]","[빠르다, 재구매]"
9,"[플레인, 상큼]","[추천, 필수]","[음료, 최고]","[빠르다, 도착]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 칼로리]","[가격, 칼로리]","[탄산, 다이어트]","[배송, 나랑드사이다]"
1,"[맛, 제로]","[가격, 사용]","[탄산, 캔]","[배송, 사이다]"
2,"[맛, 사용]","[가격, 도움]","[탄산, 부담없음]","[배송, 칼로리]"
3,"[칼로리, 부담없음]","[칼로리, 부담없음]","[다이어트, 탄산음료]","[나랑드사이다, 제로]"
4,"[칼로리, 탄산]","[칼로리, 탄산]","[다이어트, 최고]","[나랑드사이다, 맛]"
5,"[칼로리, 걱정]","[칼로리, 걱정]","[다이어트, 필수]","[나랑드사이다, 탄산]"
6,"[부담없음, 설탕]","[부담없음, 설탕]","[탄산음료, 설탕]","[제로, 부담없음]"
7,"[부담없음, 탄산음료]","[부담없음, 탄산음료]","[탄산음료, 걱정]","[제로, 음료]"
8,"[탄산, 다이어트]","[탄산, 다이어트]","[최고, 탄산수]","[맛, 포장]"
9,"[탄산, 음료]","[탄산, 음료]","[최고, 도움]","[맛, 재구매]"


,맛_review,가격_review,탄산_review,배송_review
0,"[맛, 청포도]","[가격, 저렴하다]","[탄산, 청포도]","[배송, 박스]"
1,"[맛, 파인애플]","[가격, 청포도]","[탄산, 탄산음료]","[배송, 청포도]"
2,"[맛, 레몬]","[가격, 박스]","[탄산, 콜라]","[배송, 처음]"
3,"[청포도, 복숭아]","[저렴하다, 마음]","[청포도, 파인애플]","[박스, 포장]"
4,"[청포도, 자몽]","[저렴하다, 최고]","[청포도, 복숭아]","[박스, 파인애플]"
5,"[청포도, 처음]","[저렴하다, 음료]","[청포도, 레몬]","[박스, 택배]"
6,"[복숭아, 배]","[마음, 느낌]","[파인애플, 자몽]","[포장, 라임]"
7,"[복숭아, 개인]","[마음, 상큼]","[파인애플, 배]","[포장, 도착]"
8,"[자몽, 라임]","[최고, 가성비]","[복숭아, 개인]","[파인애플, 복숭아]"
9,"[자몽, 플레인]","[최고, 파인애플]","[복숭아, 여러가지]","[파인애플, 자몽]"


In [ ]:
# 데이터프레임을 엑셀로 저장하여 후에 다시 데이터를 불러올 때 재생성하는 일이 없도록 한다.
#review_01_lst
for disp in file_lst : 
  globals()[f'{disp}_input_DataFrame'].to_excel(f'/content/drive/MyDrive/crawling_data/real_new_input_data_frm/{disp}_input_DataFrame.xlsx' , index = False)

## 결과 확인해보기

In [ ]:
all_input_data_02[9]

In [ ]:
all_input_data_02[8]

In [ ]:
all_input_data_02[7]

In [ ]:
all_input_data_02[6]

In [ ]:
all_input_data_02[5]

In [ ]:
all_input_data_02[4]

In [ ]:
all_input_data_02[3]

In [ ]:
all_input_data_02[2]

In [ ]:
all_input_data_02[1]

In [ ]:
all_input_data_02[0]

In [ ]:
file_lst

['Minemine',
 'WelchsZ',
 'ChilsungCiderZ',
 'CocacolaZ',
 'NobrandSparkling',
 'PepsiColaZ',
 'BurrZcola',
 'Reinwasser',
 'Seagram',
 'OneamSparkling',
 'BurrZcider',
 'SpriteZ',
 'Trevi',
 'NarangdCider',
 'Victoria']

In [ ]:
# 결과값을 한 엑셀에 시트별로 담는 과정이다 아래와 다른 점은 이 코드에는 cnt (빈도수)가 담겨있다.


address = '/content/drive/MyDrive/crawling_data/input_data_frm/'

with pd.ExcelWriter(address+'input_data.xlsx') as writer:
  for frm , name in zip(view_lst , file_lst) :
    x_lst = [] 
    y_lst = []
    z_lst = []
    for lst in frm :
      x_lst.append(lst[0])
      y_lst.append(lst[1])
      z_lst.append(lst[2])
    globals()[f'{name}_frm'] = pd.DataFrame({'review01' : x_lst , 'review02' : y_lst , 'cnt' : z_lst})
    globals()[f'{name}_frm'].to_excel(writer, sheet_name=f'{name}_seet' , index = False)

In [ ]:
address = '/content/drive/MyDrive/crawling_data/input_data_frm/'

with pd.ExcelWriter(address+'count_data.xlsx') as writer:
  for name in file_lst :
    
    c = Counter(globals()[f'{name}_frm'].review02)
    # print(globals()[f'{name}_frm'])
    l_01 = []
    l_02 = []

    for i in c.most_common(len(c)) :
      l_01.append(i[0])
      l_02.append(i[1])
    globals()[f'count_{name}_frm'] = pd.DataFrame({'word' : l_01 , 'cnt' : l_02 })
    globals()[f'count_{name}_frm'].to_excel(writer, sheet_name=f'{name}_seet' , index = False)


  #c = Counter(Victoria_frm.review01)
  #print(c.most_common(len(c)))

In [ ]:
c = Counter(Victoria_frm.review02)
print(c.most_common(len(c)))

[('파인애플', 11), ('가성비', 11), ('청포도', 9), ('빠르다', 9), ('재구매', 7), ('박스', 7), ('음료', 7), ('자몽', 6), ('처음', 6), ('레몬', 5), ('제품', 4), ('가격', 4), ('복숭아', 4), ('포장', 4), ('저렴하다', 3), ('마음', 3), ('택배', 2), ('최고', 2), ('추천', 2), ('구매', 1), ('탄산', 1), ('배송', 1), ('배', 1), ('라임', 1), ('도착', 1), ('유통기한', 1), ('해', 1), ('탄산음료', 1), ('콜라', 1), ('사이다', 1), ('플레인', 1), ('물', 1), ('느낌', 1)]


### 결과값

In [ ]:
# 결과 확인해보기

for i in all_input_data :
  print(i)

[['맛', '레몬'], ['맛', '빠르다'], ['맛', '재구매'], ['맛', '플레인'], ['맛', '가성비'], ['레몬', '플레인'], ['레몬', '라임'], ['레몬', '청'], ['레몬', '품절'], ['레몬', '여름'], ['빠르다', '포장'], ['빠르다', '가성비'], ['빠르다', '유통기한'], ['빠르다', '재구매'], ['빠르다', '스파클링'], ['재구매', '의사'], ['재구매', '가성비'], ['재구매', '여름'], ['재구매', '동원'], ['재구매', '해'], ['플레인', '라임'], ['플레인', '품절'], ['플레인', '유통기한'], ['플레인', '처음'], ['플레인', '에이드'], ['가성비', '최고'], ['가성비', '짱'], ['가성비', '동원'], ['가성비', '플레인'], ['가성비', '느낌'], ['배송', '빠르다'], ['배송', '제품'], ['배송', '구매'], ['배송', '탄산수'], ['배송', '재구매'], ['빠르다', '포장'], ['빠르다', '가성비'], ['빠르다', '유통기한'], ['빠르다', '재구매'], ['빠르다', '스파클링'], ['제품', '빠르다'], ['제품', '맛'], ['제품', '가성비'], ['제품', '재구매'], ['제품', '사용'], ['구매', '탄산수'], ['구매', '가격'], ['구매', '제품'], ['구매', '맛'], ['구매', '빠르다'], ['탄산수', '맛'], ['탄산수', '가격'], ['탄산수', '동원'], ['탄산수', '최고'], ['탄산수', '생수'], ['재구매', '의사'], ['재구매', '가성비'], ['재구매', '여름'], ['재구매', '동원'], ['재구매', '해'], ['가격', '제품'], ['가격', '빠르다'], ['가격', '재구매'], ['가격', '최고'], ['가격', '동원'], ['제품', '빠르다'], ['제품', '맛'], ['제품'

In [ ]:
all_input_data[14]

[['맛', '구매'],
 ['맛', '청포도'],
 ['맛', '파인애플'],
 ['맛', '제품'],
 ['맛', '재구매'],
 ['구매', '탄산'],
 ['구매', '배송'],
 ['구매', '제품'],
 ['구매', '청포도'],
 ['구매', '가격'],
 ['청포도', '파인애플'],
 ['청포도', '복숭아'],
 ['청포도', '자몽'],
 ['청포도', '레몬'],
 ['청포도', '처음'],
 ['파인애플', '복숭아'],
 ['파인애플', '자몽'],
 ['파인애플', '배'],
 ['파인애플', '레몬'],
 ['파인애플', '처음'],
 ['제품', '빠르다'],
 ['제품', '가격'],
 ['제품', '재구매'],
 ['제품', '가성비'],
 ['제품', '박스'],
 ['재구매', '청포도'],
 ['재구매', '빠르다'],
 ['재구매', '가성비'],
 ['재구매', '파인애플'],
 ['재구매', '음료'],
 ['배송', '빠르다'],
 ['배송', '제품'],
 ['배송', '박스'],
 ['배송', '포장'],
 ['배송', '재구매'],
 ['빠르다', '포장'],
 ['빠르다', '저렴하다'],
 ['빠르다', '가성비'],
 ['빠르다', '청포도'],
 ['빠르다', '마음'],
 ['제품', '빠르다'],
 ['제품', '가격'],
 ['제품', '재구매'],
 ['제품', '가성비'],
 ['제품', '박스'],
 ['박스', '포장'],
 ['박스', '택배'],
 ['박스', '파인애플'],
 ['박스', '레몬'],
 ['박스', '처음'],
 ['포장', '라임'],
 ['포장', '도착'],
 ['포장', '택배'],
 ['포장', '유통기한'],
 ['포장', '비비'],
 ['재구매', '청포도'],
 ['재구매', '빠르다'],
 ['재구매', '가성비'],
 ['재구매', '파인애플'],
 ['재구매', '음료'],
 ['가격', '저렴하다'],
 ['가격', '재구매'],
 ['가격', 